In [16]:
llm_config = {"model": "gpt-3.5-turbo"}

In [3]:
task = '''
        Write a concise but engaging blogpost about
       Friend to become a Wife. Make sure the blogpost is
       within 100 words.
       '''

In [5]:
from autogen.agentchat.assistant_agent import AssistantAgent
from autogen.agentchat.chat import initiate_chats

In [7]:
writer = AssistantAgent(
    name='writer',
    system_message = "You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [8]:
writer.generate_reply(messages=[{'content':task,'role':'user'}])

"Title: From Friend to Life Partner: A Beautiful Journey\n\nEmbarking on a romantic relationship with a friend can be a wonderfully fulfilling experience. You already share a deep connection and mutual understanding, laying a strong foundation for a lasting bond. The transition from friends to spouses can bring a new level of intimacy and companionship, enhancing the relationship in ways you might have never imagined. This beautiful journey allows you to appreciate each other's quirks, support one another through thick and thin, and build a love that stems from a strong friendship. So, cherish the evolution from friend to life partner – it's truly special."

In [9]:
critic = AssistantAgent(
    name='critic',
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [10]:
critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to writer):


        Write a concise but engaging blogpost about
       Friend to become a Wife. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
writer (to critic):

Title: From Friend to Life Partner: A Beautiful Journey

Embarking on a romantic relationship with a friend can be a wonderfully fulfilling experience. You already share a deep connection and mutual understanding, laying a strong foundation for a lasting bond. The transition from friends to spouses can bring a new level of intimacy and companionship, enhancing the relationship in ways you might have never imagined. This beautiful journey allows you to appreciate each other's quirks, support one another through thick and thin, and build a love that stems from a strong friendship. So, cherish the evolution from friend to life partner – it's truly special.

---------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': '\n        Write a concise but engaging blogpost about\n       Friend to become a Wife. Make sure the blogpost is\n       within 100 words.\n       ', 'role': 'assistant'}, {'content': "Title: From Friend to Life Partner: A Beautiful Journey\n\nEmbarking on a romantic relationship with a friend can be a wonderfully fulfilling experience. You already share a deep connection and mutual understanding, laying a strong foundation for a lasting bond. The transition from friends to spouses can bring a new level of intimacy and companionship, enhancing the relationship in ways you might have never imagined. This beautiful journey allows you to appreciate each other's quirks, support one another through thick and thin, and build a love that stems from a strong friendship. So, cherish the evolution from friend to life partner – it's truly special.", 'role': 'user'}, {'content': 'This blogpost effectively captures the essence of transitioning fro

In [12]:
SEO_reviewer = AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


legal_reviewer = AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

ethics_reviewer = AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer
)

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to writer):


        Write a concise but engaging blogpost about
       Friend to become a Wife. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
writer (to critic):

Title: From Friend to Life Partner: A Beautiful Journey

Embarking on a romantic relationship with a friend can be a wonderfully fulfilling experience. You already share a deep connection and mutual understanding, laying a strong foundation for a lasting bond. The transition from friends to spouses can bring a new level of intimacy and companionship, enhancing the relationship in ways you might have never imagined. This beautiful journey allows you to appreciate each other's quirks, support one another through thick and thin, and build a love that stems from a strong friendship. So, cherish the evolution from friend to life partner – it's truly special.

---------------------------------------------------------------------